In [9]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from fundamental_library import *
from digital_twin_library import ConvModel, train_epoch, get_correlations, GaussianReadout
from neuralpredictors.measures.modules import PoissonLoss 

# Set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the root directory
root_dir = '/project/data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6/data'

# Initialize lists to collect data
import os

# Initialize lists to collect data
responses_list = []
images_list = []

# Load the data
for n in range(5994):
    image_path = os.path.join(root_dir, 'images', f'{n}.npy')
    response_path = os.path.join(root_dir, 'responses', f'{n}.npy')
    
    response_data = np.load(response_path)
    image_data = np.load(image_path)
    
    # Ensure image is grayscale
    if image_data.ndim == 3 and image_data.shape[0] == 3:
        image_data = np.mean(image_data, axis=0, keepdims=True)
    
    responses_list.append(response_data)
    images_list.append(image_data)
    
# Convert lists to NumPy arrays
sensorium_responses = np.array(responses_list)
sensorium_images = np.array(images_list)

# Optionally convert to PyTorch tensors
sensorium_responses_tensor = torch.tensor(sensorium_responses, device=device)
sensorium_images_tensor = torch.tensor(sensorium_images, device=device)

# Apply the custom transform
pretrain_transform = CustomTransform()
transformed_sensorium_images = torch.stack([pretrain_transform(img) for img in sensorium_images_tensor]).squeeze()
print(transformed_sensorium_images.shape)
transformed_sensorium_images_np = transformed_sensorium_images.cpu().numpy().astype('uint8')
print(f'transformed images in np shape {transformed_sensorium_images_np.shape}')

pretrain_train_ratio=0.8
pretrain_val_ratio=0.1
pretrain_dataset = NeuralDataset(transformed_sensorium_images_np, sensorium_responses_tensor)
pretrain_total_size = len(pretrain_dataset)
pretrain_train_size = int(pretrain_train_ratio * pretrain_total_size)
pretrain_val_size = int(pretrain_val_ratio * pretrain_total_size)
pretrain_test_size = pretrain_total_size - pretrain_train_size - pretrain_val_size
pretrain_train_dataset, val_dataset, test_dataset = random_split(pretrain_dataset, [pretrain_train_size, pretrain_val_size, pretrain_test_size])
pretrain_train_loader = DataLoader(pretrain_train_dataset, batch_size=32, shuffle=True)
pretrain_val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
pretrain_test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'shape in dataloader {next(iter(pretrain_train_loader))[0].shape}')

# Define early stopping criteria
early_stopping_patience = 5
early_stopping_counter = 0
best_val_loss = float('-inf')

# Initialize the model for pretraining
pretrain_model = ConvModel(
    layers=7, 
    input_kern=11,
    hidden_kern=5, 
    hidden_channels=64, 
    output_dim=8372,  # Output dimension for the pretraining dataset
    std_scale=0.5
)

pretrain_model = pretrain_model.to(device)
poisson_loss = PoissonLoss() # use different loss
gamma = 1e-2
loss_fn = lambda outputs, targets: poisson_loss(outputs, targets) + gamma * pretrain_model.regularizer()

optimizer = torch.optim.Adam(pretrain_model.parameters(), lr=1e-3)

# Define the learning rate schedule
lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

# Define the number of epochs
epochs = 100

for epoch in range(epochs):
    # Training loop
    loss = my_train_epoch(pretrain_model, pretrain_train_loader, optimizer, loss_fn, device)
    
    # Validation loop
    with torch.no_grad():
        val_corrs = get_correlations(pretrain_model, pretrain_val_loader, device)
    validation_correlation = val_corrs.mean()
    
    # Update learning rate schedule
    lr_scheduler.step(validation_correlation)
    
    # Print training and validation losses
    print(f'Epoch [{epoch+1}/{epochs}], validation correlation: {validation_correlation:.4f}, trainloss: {loss:.4f}')
    
    # Check for early stopping
    if validation_correlation > best_val_loss:
        best_val_loss = validation_correlation
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_patience:
            print('Early stopping triggered!')
            break

print('Freezing core.')

for param in pretrain_model.core.parameters():
    param.requires_grad = False



torch.Size([5994, 64, 64])
transformed images in np shape (5994, 64, 64)
shape in dataloader torch.Size([32, 1, 64, 64])
Epoch [1/100], validation correlation: 0.0386, trainloss: 0.8186
Epoch [2/100], validation correlation: 0.0602, trainloss: 0.6160
Epoch [3/100], validation correlation: 0.0760, trainloss: 0.5696
Epoch [4/100], validation correlation: 0.0781, trainloss: 0.5534
Epoch [5/100], validation correlation: 0.0951, trainloss: 0.5396


In [4]:
# def get_last_conv_out_channels(model):
#     for layer in reversed(model.core):
#         if isinstance(layer, nn.Conv2d):
#             return layer.out_channels
#     raise ValueError("No Conv2d layer found in the model.")

# # Retrieve the number of output channels from the last Conv2d layer
# hidden_channels = get_last_conv_out_channels(pretrain_model)

pretrain_model.readout = GaussianReadout(13, 64,spatial_scale=0.05, std_scale=0.8).to(device)

In [7]:
images_path = '/project/subiculum/data/images_uint8.npy'
v1_responses_path = '/project/subiculum/data/V1_Data.mat'

# Load Images to np.array
images=np.load(images_path)
print(images.shape)
# Load responses and preprocess them

v1_responses,_,_ = load_mat_file(v1_responses_path)
v1_responses = preprocess_responses(v1_responses)

#define Dataset
train_ratio=0.6
val_ratio=0.2

dataset = NeuralDataset(images, v1_responses)
total_size = len(dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Redefine the loss function and optimizer for the new training
loss_fn = PoissonLoss()
optimizer = torch.optim.Adam(pretrain_model.readout.parameters(), lr=1e-3)

# Training loop for the new dataset
def train_readout(model, train_loader, val_loader, num_epochs, optimizer, loss_fn, device):
    model.eval()  # Make sure the core is in eval mode
    model.readout.train()  # Only train the readout

    early_stopping_patience = 5
    early_stopping_counter = 0
    best_val_loss = float('-inf')
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

    for epoch in range(num_epochs):
        model.readout.train()
        train_loss = 0.0
        for images, responses in train_loader:
            images, responses = images.to(device), responses.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, responses)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        val_loss = 0.0
        model.readout.eval()
        with torch.no_grad():
            for images, responses in val_loader:
                images, responses = images.to(device), responses.to(device)
                outputs = model(images)
                loss = loss_fn(outputs, responses)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        with torch.no_grad():
            val_corrs = get_correlations(model, val_loader, device)
            validation_correlation = val_corrs.mean()

        lr_scheduler.step(val_loss)

        print(f'Epoch [{epoch+1}/{epochs}], validation correlation: {validation_correlation:.4f}, trainloss: {train_loss:.4f}')

        if validation_correlation > best_val_loss:
            best_val_loss = validation_correlation
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= early_stopping_patience:
                print('Early stopping triggered!')
                break

# Train the readout
train_readout(pretrain_model, train_loader, val_loader, num_epochs=100, optimizer=optimizer, loss_fn=loss_fn, device=device)

(2500, 1200, 1200)


/usr/local/lib/python3.8/dist-packages/neuralpredictors/measures/modules.py:78: UserWarning: Poissonloss is averaged per batch. It's recommended to use `sum` instead
  warnings.warn("Poissonloss is averaged per batch. It's recommended to use `sum` instead")


Epoch [1/100], validation correlation: 0.0058, trainloss: 0.3498
Epoch [2/100], validation correlation: 0.0100, trainloss: 0.3230
Epoch [3/100], validation correlation: 0.0155, trainloss: 0.3135
Epoch [4/100], validation correlation: 0.0177, trainloss: 0.3083
Epoch     5: reducing learning rate of group 0 to 1.0000e-04.
Epoch [5/100], validation correlation: 0.0245, trainloss: 0.3032
Epoch [6/100], validation correlation: 0.0246, trainloss: 0.3042
Epoch [7/100], validation correlation: 0.0248, trainloss: 0.3017
Epoch [8/100], validation correlation: 0.0251, trainloss: 0.3018
Epoch     9: reducing learning rate of group 0 to 1.0000e-05.
Epoch [9/100], validation correlation: 0.0250, trainloss: 0.2992
Epoch [10/100], validation correlation: 0.0250, trainloss: 0.3004
Epoch [11/100], validation correlation: 0.0250, trainloss: 0.3007
Epoch [12/100], validation correlation: 0.0250, trainloss: 0.3003
Epoch    13: reducing learning rate of group 0 to 1.0000e-06.
Epoch [13/100], validation corr

In [8]:
# Evaluation
def evaluate_model(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    all_preds = []
    all_resps = []
    loss_fn = nn.PoissonNLLLoss(log_input=False)
    
    with torch.no_grad():
        for images, responses in test_loader:
            images, responses = images.to(device), responses.to(device)  # Move data to the appropriate device
            outputs = model(images)
            loss = loss_fn(outputs, responses)
            test_loss += loss.item()
            all_preds.append(outputs.cpu().numpy())
            all_resps.append(responses.cpu().numpy())
    
    test_loss /= len(test_loader)
    print(f'Test Loss: {test_loss:.4f}')
    
    all_preds = np.vstack(all_preds)
    all_resps = np.vstack(all_resps)
    correlation = get_correlations(model, test_loader, device)
    print(f'Test Correlation: {correlation.mean():.4f}')  # Print mean correlation
    
    return all_preds, all_resps

# Evaluate the model

evaluate_model(pretrain_model, test_loader, device)

Test Loss: 0.3071
Test Correlation: 0.0191


(array([[4.32638749e-02, 1.33319855e-01, 4.57171760e-02, ...,
         2.66365558e-01, 8.71798471e-02, 1.03673629e-01],
        [5.54040354e-03, 4.20112535e-02, 1.72401350e-02, ...,
         2.16302261e-01, 3.12626585e-02, 5.66082858e-02],
        [1.66791439e-01, 1.89986199e-01, 1.55536070e-01, ...,
         2.14892358e-01, 1.73058018e-01, 1.77892044e-01],
        ...,
        [2.19548983e-03, 6.52281195e-02, 7.33534619e-02, ...,
         3.08219582e-01, 1.34813255e-02, 2.27863565e-02],
        [1.97051835e-04, 1.72889903e-02, 2.10766541e-03, ...,
         1.71947032e-01, 5.69103332e-03, 1.11703649e-02],
        [9.06157866e-02, 2.15883121e-01, 8.96869227e-02, ...,
         2.77653307e-01, 1.13274664e-01, 1.18021294e-01]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 2., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dt